In [7]:
import pandas as pd
from datetime import datetime,timedelta
from prophet import Prophet
import os

In [8]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho)

In [9]:
tbl_ = pd.read_excel(path_arq+'\\cubo_anl\\tbl_full_analitica.xlsx')
tbl_2  = tbl_[['CHAMADO','DT_ABERTURA_CHAMADO','MODULO_CHAMADO','TIPO_CHAMADO','COMPLEXIDADE']]

In [10]:
# Convertendo a coluna de data para o formato datetime
tbl_2['DT_ABERTURA_CHAMADO'] = tbl_2['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')

C:\Users\andre\AppData\Local\Temp\ipykernel_10940\193842085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_2['DT_ABERTURA_CHAMADO'] = tbl_2['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')


In [11]:
# Pega data minima da amostra:

dt_max = tbl_2['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]').max()
dt_min = (dt_max - timedelta(days=20)).strftime('%Y-%m-%d')

In [12]:
#Agrupando e contando volume de entradas por modulo, tipo e complexidade diariamente:
df_agrupado = tbl_2.groupby(['DT_ABERTURA_CHAMADO']).agg({'CHAMADO':'count'}).reset_index().sort_values(['DT_ABERTURA_CHAMADO'])

In [13]:
#Filtrando a partir da data minima:
df_agrupado = df_agrupado[df_agrupado['DT_ABERTURA_CHAMADO']>= dt_min]
df_agrupado['DT_ABERTURA_CHAMADO'] = df_agrupado['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]')

In [14]:
df_prophet = df_agrupado[['DT_ABERTURA_CHAMADO', 'CHAMADO']].rename(columns={'DT_ABERTURA_CHAMADO': 'ds', 'CHAMADO': 'y'})

In [15]:
model = Prophet(changepoint_prior_scale=0.5, seasonality_mode='additive')
model.fit(df_prophet)

21:12:02 - cmdstanpy - INFO - Chain [1] start processing
21:12:02 - cmdstanpy - INFO - Chain [1] done processing


In [16]:
dt_max.strftime('%Y-%m-%d')

'2024-04-01'

In [17]:
intervalo_datas = pd.date_range(start=dt_max, periods=30, freq='D')
proximos_30_dias = intervalo_datas[intervalo_datas.weekday < 5][:30] 

previsao_futura = pd.DataFrame({'ds': proximos_30_dias})
forecast = model.predict(previsao_futura)

In [18]:
# Filtrando as previsões a partir de 2024-04-02
previsao_futura = forecast[forecast['ds'] > dt_max]

# Exibindo as previsões para o período desejado
previsao_futura = previsao_futura[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

#criando ponderados que faz a média entre a média mais o desvio com a média prevista
previsao_futura['previsao'] = round(previsao_futura['yhat'],0)

In [19]:
previsao_futura[['ds','previsao']].to_excel(path_arq+'//modelo//Previsao_demandas.xlsx',index=False)